# ZKP positive number proof

In [7]:
from Crypto.Util import number
import numpy as np

In [8]:
def ZKPrandom(bitlength=8):
    x = [number.getRandomNBitInteger(8) for i in range(4)]
    return x, int(np.sum(x))

def ZKPnumber(bitlength=4):
    x = [number.getRandomNBitInteger(bitlength) ** 2 for i in range(4)]
    return x, int(np.sum(x))

def ZKPpositive(x, g=3, h=5, n=pow(2,19)-1):
    c_0, c_1, c_2, c_3 = [(pow(g, x[i], n) * pow(h, r[i], n)) % n for i in range(4)]
    c = (c_0 * c_1 * c_2 * c_3) % n
    return c

In [9]:
# Alice and Bob agree on bases g,h and prime n
g, h, n = 3, 5, pow(2,19)-1

x, x_sum = ZKPnumber(4)
r, r_sum = ZKPrandom(8)

print("Alice has number %d and generates random shares" % (x_sum),r)

x_minus_a, x_minus_a_sum = ZKPnumber(2)
b_minus_x, b_minus_x_sum = ZKPnumber(6)

print("Range is [%d,%d]" % (x_sum - x_minus_a_sum, b_minus_x_sum + x_sum))

ZKPpositive(x_minus_a) == pow(g,x_minus_a_sum,n) * pow(h,r_sum,n) % n, ZKPpositive(b_minus_x) == pow(g,b_minus_x_sum,n) * pow(h,r_sum,n) % n

Alice has number 531 and generates random shares [229, 247, 240, 192]
Range is [515,9368]


(True, True)

In [16]:
from ecpy.curves import Curve
import random

In [17]:
def curvePick(x):
    """
    ['stark256',
 'frp256v1',
 'secp521r1',
 'secp384r1',
 'secp256k1',
 'secp256r1',
 'secp224k1',
 'secp224r1',
 'secp192k1',
 'secp192r1',
 'secp160k1',
 'secp160r1',
 'secp160r2',
 'Brainpool-p512t1',
 'Brainpool-p512r1',
 'Brainpool-p384t1',
 'Brainpool-p384r1',
 'Brainpool-p320t1',
 'Brainpool-p320r1',
 'Brainpool-p256r1',
 'Brainpool-p256t1',
 'Brainpool-p224r1',
 'Brainpool-p224t1',
 'Brainpool-p192r1',
 'Brainpool-p192t1',
 'Brainpool-p160r1',
 'Brainpool-p160t1',
 'NIST-P256',
 'NIST-P224',
 'NIST-P192',
 'Ed448',
 'Ed25519',
 'Curve448',
 'Curve25519']
    """
    curve = Curve.get_curve(Curve.get_curve_names()[x])
    return curve

In [18]:
curve = curvePick(4) # 4 is secp256k1
G = curve.generator
order = curve.order
a = [1, 44, 33, 10]
b = [60, 54, 19, 10]

assert all(a[i] > 0 for i in range(len(a))), "Must use positive numbers"
assert all(b[i] > 0 for i in range(len(b))), "Must use positive numbers"
 

s = number.getRandomNBitInteger(256)
r = number.getRandomNBitInteger(256)

raAi = [r * a[i] * G for i in range(len(a))]
sraAi = [s * raAi[i] for i in range(len(a))]

inv_r = pow(r, -1, order)

random.shuffle(sraAi)

sAi = [inv_r * sraAi[i] for i in range(len(a))]  
sBi = [s * b[i] * G for i in range(len(b))]
    
len(np.intersect1d([sBi[i].x for i in range(len(a))], [sAi[i].x for i in range(len(a))]))

1